In [1]:
import pandas as pd
import numpy as np

import string  
import nltk
import re
import scipy
import matplotlib.pyplot as plt
import urllib 
import sklearn
from sklearn import preprocessing, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
display(train)
display(test)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [2]:
train['keyword'].fillna('no keyword', inplace = True) 
train['keyword'] = train['keyword'].str.replace('%20', ' ')
train['location'].fillna('no location', inplace = True)

test['keyword'].fillna('no keyword', inplace = True) 
test['keyword'] = test['keyword'].str.replace('%20', ' ')
test['location'].fillna('no location', inplace = True)

train['length'] = train['text'].str.len()
test['length'] = test['text'].str.len()

In [3]:
#limpiando el texto

import nltk
#nltk.download('stopwords') 
#nltk.download('punkt')
#nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#FUNCIONES UTILES

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

#Cleaning text

train['clean_text'] = train['text'].str.lower()

train['clean_text'] = train['clean_text'].apply(only_letters)    

test['clean_text'] = test['text'].str.lower()

test['clean_text'] = test['clean_text'].apply(only_letters) 

#Tokenización

train['clean_text'] = train['clean_text'].apply(word_tokenize)

test['clean_text'] = test['clean_text'].apply(word_tokenize)

#Remove stopwords

train['clean_text'] = train['clean_text'].apply(filter_stopwords) 

test['clean_text'] = test['clean_text'].apply(filter_stopwords) 


#Lemmatization                                                                       

train['clean_text'] = train['clean_text'].apply(lemmatize_tweet)  

train['clean_text'] = train['clean_text'].apply(transform_to_text)

test['clean_text'] = test['clean_text'].apply(lemmatize_tweet)  

test['clean_text'] = test['clean_text'].apply(transform_to_text)

display(train)

,id,keyword,location,text,target,length,clean_text
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69,deed reason earthquake may allah forgive u
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38,forest fire near la ronge sask canada
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133,resident asked shelter place notified officer ...
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65,people receive wildfire evacuation order calif...
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88,got sent photo ruby alaska smoke wildfire pour...
...,...,...,...,...,...,...,...
7608,10869,no keyword,no location,Two giant cranes holding a bridge collapse int...,1,83,two giant crane holding bridge collapse nearby...
7609,10870,no keyword,no location,@aria_ahrary @TheTawniest The out of control w...,1,125,ariaahrary thetawniest control wild fire calif...
7610,10871,no keyword,no location,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65,utckm volcano hawaii
7611,10872,no keyword,no location,Police investigating after an e-bike collided ...,1,137,police investigating ebike collided car little...


In [4]:
#agregando algunos features

keywords_list = list(set(train.keyword))  #todas las keywords únicas

locations_list = list(set(train.location)) #todas las locaciones únicas
locations_list.remove('no location')

def amount_keywords(tweet):
    cont = 0
    for keyword in keywords_list:
        if keyword in tweet:
            cont += 1
    return cont

train['amount_words'] = train['clean_text'].str.split(' ').apply(lambda x: len(x))  

test['amount_words'] = test['clean_text'].str.split(' ').apply(lambda x: len(x))  


train['amount_keywords_in_tweet'] = train['clean_text'].apply(lambda x: amount_keywords(x))

test['amount_keywords_in_tweet'] = test['clean_text'].apply(lambda x: amount_keywords(x))


tiene_mas_de_un_kw = train[train['amount_keywords_in_tweet'] > 1]

tiene_un_solo_kw = train[train['amount_keywords_in_tweet'] == 1]

no_tiene_kw = train[train['amount_keywords_in_tweet'] == 0]

porcentaje_target_mas_de_un_kw = tiene_mas_de_un_kw['target'].mean()
print("porcentaje target de tweets con mas de un keyword", porcentaje_target_mas_de_un_kw)

porcentaje_target_un_solo_kw = tiene_un_solo_kw['target'].mean()
print("porcentaje target de tweets con un solo keyword", porcentaje_target_un_solo_kw)

porcentaje_target_sin_kw = no_tiene_kw['target'].mean()
print("porcentaje target de tweets sin keyword", porcentaje_target_sin_kw)

print("el maximo de cant de keywords en un tweet es", tiene_mas_de_un_kw['amount_keywords_in_tweet'].max())

def amount_locations(tweet):    #esta función podría unificarse con la de keywords pasandole por parámetro la lista a recorrer
    cont = 0
    for location in locations_list:
        if location in tweet:
            cont += 1
    return cont

train['amount_locations_in_tweet'] = train['clean_text'].apply(lambda x: amount_locations(x))

test['amount_locations_in_tweet'] = test['clean_text'].apply(lambda x: amount_locations(x))


mas_de_1_location = train[train['amount_locations_in_tweet'] > 1]

un_solo_location = train[train['amount_locations_in_tweet'] == 1]

no_tiene_location = train[train['amount_locations_in_tweet'] == 0]

porcentaje_target_mas_de_un_location = mas_de_1_location['target'].mean()
print("porcentaje target de tweets con mas de una locacion", porcentaje_target_mas_de_un_location)

porcentaje_target_un_solo_location = un_solo_location['target'].mean()
print("porcentaje target de tweets con una sola locacion", porcentaje_target_un_solo_location)

porcentaje_target_sin_location = no_tiene_location['target'].mean()
print("porcentaje target de tweets sin locacion", porcentaje_target_sin_location)

print("el maximo de cant de locaciones en un tweet es", mas_de_1_location['amount_locations_in_tweet'].max())

porcentaje target de tweets con mas de un kw 0.606816584680253
porcentaje target de tweets con un solo kw 0.3446327683615819
porcentaje target de tweets sin kw 0.2025862068965517
el maximo de cant de keywords en un tweet es 7
porcentaje target de tweets con mas de una locacion 0.504933504933505
porcentaje target de tweets con una sola locacion 0.4489409984871407
porcentaje target de tweets sin locacion 0.34382107657316147
el maximo de cant de locaciones en un tweet es 8


In [5]:
mas_de_un_location_y_mas_de_un_keyword = train[(train['amount_locations_in_tweet'] > 1) & (train['amount_keywords_in_tweet'] > 1)]
porcentaje_target_mas_de_un_location_y_mas_de_un_keyword = mas_de_un_location_y_mas_de_un_keyword.target.mean()
porcentaje_target_mas_de_un_location_y_mas_de_un_keyword

0.6990196078431372

In [6]:
#más features
def get_hashtags(s):
    return list(part[1:] for part in s.split() if part.startswith('#'))

train['hashtags'] = train['text'].apply(get_hashtags)
test['hashtags'] = test['text'].apply(get_hashtags)

def hashtag_length_proportion(hashtags, length):
    return len(''.join(hashtags))/length

train['len_hash_over_text'] = train.apply(lambda data: hashtag_length_proportion(data['hashtags'], data['length']), axis=1)
test['len_hash_over_text'] = test.apply(lambda data: hashtag_length_proportion(data['hashtags'], data['length']), axis=1)

def get_mentioned_users(s):
    return list(part[1:] for part in s.split() if part.startswith('@'))

train['users'] = train['text'].apply(get_mentioned_users)
test['users'] = test['text'].apply(get_mentioned_users)

def amount_hashtags(l):
    return len(l)

train['amount_hashtags'] = train['hashtags'].apply(amount_hashtags)
test['amount_hashtags'] = test['hashtags'].apply(amount_hashtags)

def amount_users(l):
    return len(l)

def has_url(text):
    return int('http' in text)

train['has_url'] = train['text'].apply(has_url)
test['has_url'] = test['text'].apply(has_url)

def find_url(text):
    urls = re.findall(r'(https?://\S+)', text)
    return urls

train['urls'] = train['text'].apply(find_url)
test['urls'] = test['text'].apply(find_url)
train['amount_urls'] = train['urls'].apply(lambda x: len(x))
test['amount_urls'] = test['urls'].apply(lambda x: len(x))

train['amount_users'] = train['users'].apply(amount_users)
test['amount_users'] = test['users'].apply(amount_users)


In [7]:
#construyendo los datos de entrenamiento y de test
selected_df = train[['length','amount_words','amount_keywords_in_tweet','amount_locations_in_tweet', 'amount_urls','amount_users', 'amount_hashtags','len_hash_over_text', 'has_url']] 

X, y = selected_df,train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 41)

In [8]:
X_train

,length,amount_words,amount_keywords_in_tweet,amount_locations_in_tweet,amount_urls,amount_users,amount_hashtags,len_hash_over_text,has_url
79,91,6,1,0,0,0,0,0.000000,0
2155,135,11,2,1,1,1,0,0.000000,1
7574,63,5,2,0,1,0,0,0.000000,1
286,139,12,1,1,0,0,0,0.000000,0
3659,140,14,3,1,0,0,0,0.000000,0
...,...,...,...,...,...,...,...,...,...
5200,139,14,4,4,0,0,2,0.208633,0
4066,103,10,2,0,1,1,0,0.000000,1
3980,90,8,2,1,1,0,0,0.000000,1
931,134,9,0,2,1,2,1,0.097015,1


**Random Forest como problema de regresión**

In [14]:
#Random forest por defecto
rf_model = RandomForestRegressor(random_state = 1)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
rf_model.feature_importances_
preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.470008
F1_score:  0.6157443491816056


In [22]:
#Random forest prueba2
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 85, max_features = 6, min_samples_split = 10, max_depth = 12,
                                min_samples_leaf = 11)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ",f1_score(y_test, preds))

RMSE: 0.441015
F1_score:  0.6159110350727117


In [18]:
#Random forest prueba3
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 80, max_features = 7, min_samples_split = 10, max_depth = 7,
                                min_samples_leaf = 25)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ",f1_score(y_test, preds))

RMSE: 0.441364
F1_score:  0.6176470588235294


In [20]:
#Random forest prueba4
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 85, max_features = 7, min_samples_split = 10, max_depth = 9,
                                min_samples_leaf = 25)
#VARIANDO MIN_SAMPLES_SPLIT NO VEO QUE CAMBIE EL RESULTADO
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ",f1_score(y_test, preds))

RMSE: 0.440260
F1_score:  0.622673434856176


In [24]:
#Random forest prueba5
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 85, max_features = 5, min_samples_split = 10, max_depth = 11,
                                min_samples_leaf = 10)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ",f1_score(y_test, preds))

RMSE: 0.441127
F1_score:  0.6235093696763203


In [19]:
#Random forest prueba6
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 80, max_features = 7, min_samples_split = 10, max_depth = 9,
                                min_samples_leaf = 25)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ",f1_score(y_test, preds))

RMSE: 0.440271
F1_score:  0.6236193712829227


In [23]:
#Random forest prueba7
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 85, max_features = 5, min_samples_split = 10, max_depth = 12,
                                min_samples_leaf = 11)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ",f1_score(y_test, preds))

RMSE: 0.440582
F1_score:  0.6257562662057045


In [21]:
#Random forest prueba8
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 85, max_features = 7, min_samples_split = 10, max_depth = 9,
                                min_samples_leaf = 11)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.442076
F1_score:  0.6258503401360545


In [25]:
#Random forest prueba9
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 87, max_features = 7, min_samples_split = 10, max_depth = 9,
                                min_samples_leaf = 11)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.442059
F1_score:  0.6263829787234043


In [26]:
#Random forest prueba10
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 90, max_features = 7, min_samples_split = 10, max_depth = 9,
                                min_samples_leaf = 11)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.442035
F1_score:  0.629251700680272


In [68]:
#Random forest prueba11
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 91, max_features = 7, min_samples_split = 8, max_depth = 9,
                                min_samples_leaf = 11)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.442067
F1_score:  0.629251700680272


**Random Forest como problema de clasificación**

In [70]:
#Random forest por defecto
rf_model = RandomForestClassifier(random_state=1)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

print("F1_score: ", f1_score(y_test, preds))

rf_model.feature_importances_

RMSE: 0.560813
F1_score:  0.6219415943172849


array([0.39084063, 0.16981721, 0.12194693, 0.08641244, 0.04240295,
       0.04769215, 0.03211079, 0.08180255, 0.02697435])

In [74]:
#Random forest prueba2
rf_model = RandomForestClassifier(random_state = 1, n_estimators = 90, max_features = 7, min_samples_split = 10, max_depth = 8,
                                min_samples_leaf = 15)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.537497
F1_score:  0.6219931271477663


In [75]:
#Random forest prueba3
rf_model = RandomForestClassifier(random_state = 1, n_estimators = 90, max_features = 7, min_samples_split = 10, max_depth = 10,
                                min_samples_leaf = 15)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.534435
F1_score:  0.6304163126593033


In [76]:
#Random forest prueba4
rf_model = RandomForestClassifier(random_state = 1, n_estimators = 85, max_features = 7, min_samples_split = 10, max_depth = 10,
                                min_samples_leaf = 15)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.533820
F1_score:  0.6322033898305085


In [82]:
#Random forest prueba5
rf_model = RandomForestClassifier(random_state = 1, n_estimators = 83, max_features = 7, min_samples_split = 10, max_depth = 10,
                                min_samples_leaf = 15)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.533205
F1_score:  0.6327396098388465


**Probando Randomized Search CV**

In [87]:
params = {
 "max_depth"        : [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 "n_estimators"     : [80, 83, 84, 85, 87, 89, 90, 91, 93],
 "min_samples_split": [10, 12, 14, 15, 16, 18],
 "min_samples_leaf" : [10, 12, 14, 15, 16, 18],
 "max_features"     : [2, 3, 4, 5, 6, 7, 8, 9]
} 

model = RandomForestClassifier(random_state = 1)

random_search = RandomizedSearchCV(model,param_distributions = params,scoring = 'f1', cv = 5,verbose = 3,n_iter = 100)
random_search.fit(X, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] n_estimators=85, min_samples_split=12, min_samples_leaf=14, max_features=3, max_depth=4 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=85, min_samples_split=12, min_samples_leaf=14, max_features=3, max_depth=4, score=0.499, total=   0.5s
[CV] n_estimators=85, min_samples_split=12, min_samples_leaf=14, max_features=3, max_depth=4 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  n_estimators=85, min_samples_split=12, min_samples_leaf=14, max_features=3, max_depth=4, score=0.470, total=   0.5s
[CV] n_estimators=85, min_samples_split=12, min_samples_leaf=14, max_features=3, max_depth=4 


...

[CV]  n_estimators=90, min_samples_split=16, min_samples_leaf=15, max_features=4, max_depth=8, score=0.561, total=   0.8s
[CV] n_estimators=90, min_samples_split=16, min_samples_leaf=15, max_features=4, max_depth=8 
[CV]  n_estimators=90, min_samples_split=16, min_samples_leaf=15, max_features=4, max_depth=8, score=0.543, total=   0.8s
[CV] n_estimators=90, min_samples_split=16, min_samples_leaf=15, max_features=4, max_depth=8 
[CV]  n_estimators=90, min_samples_split=16, min_samples_leaf=15, max_features=4, max_depth=8, score=0.605, total=   0.7s
[CV] n_estimators=90, min_samples_split=16, min_samples_leaf=15, max_features=4, max_depth=8 
[CV]  n_estimators=90, min_samples_split=16, min_samples_leaf=15, max_features=4, max_depth=8, score=0.580, total=   0.7s
[CV] n_estimators=90, min_samples_split=16, min_samples_leaf=15, max_features=4, max_depth=8 
[CV]  n_estimators=90, min_samples_split=16, min_samples_leaf=15, max_features=4, max_depth=8, score=0.640, total=   0.7s
[CV] n_estimat

[CV]  n_estimators=93, min_samples_split=14, min_samples_leaf=12, max_features=7, max_depth=8, score=0.581, total=   0.9s
[CV] n_estimators=93, min_samples_split=14, min_samples_leaf=12, max_features=7, max_depth=8 
[CV]  n_estimators=93, min_samples_split=14, min_samples_leaf=12, max_features=7, max_depth=8, score=0.644, total=   0.9s
[CV] n_estimators=83, min_samples_split=12, min_samples_leaf=12, max_features=9, max_depth=6 
[CV]  n_estimators=83, min_samples_split=12, min_samples_leaf=12, max_features=9, max_depth=6, score=0.545, total=   0.8s
[CV] n_estimators=83, min_samples_split=12, min_samples_leaf=12, max_features=9, max_depth=6 
[CV]  n_estimators=83, min_samples_split=12, min_samples_leaf=12, max_features=9, max_depth=6, score=0.498, total=   0.8s
[CV] n_estimators=83, min_samples_split=12, min_samples_leaf=12, max_features=9, max_depth=6 
[CV]  n_estimators=83, min_samples_split=12, min_samples_leaf=12, max_features=9, max_depth=6, score=0.602, total=   0.9s
[CV] n_estimat

[CV]  n_estimators=89, min_samples_split=16, min_samples_leaf=14, max_features=9, max_depth=3, score=0.478, total=   0.6s
[CV] n_estimators=89, min_samples_split=16, min_samples_leaf=14, max_features=9, max_depth=3 
[CV]  n_estimators=89, min_samples_split=16, min_samples_leaf=14, max_features=9, max_depth=3, score=0.540, total=   0.6s
[CV] n_estimators=89, min_samples_split=16, min_samples_leaf=14, max_features=9, max_depth=3 
[CV]  n_estimators=89, min_samples_split=16, min_samples_leaf=14, max_features=9, max_depth=3, score=0.492, total=   0.6s
[CV] n_estimators=89, min_samples_split=16, min_samples_leaf=14, max_features=9, max_depth=3 
[CV]  n_estimators=89, min_samples_split=16, min_samples_leaf=14, max_features=9, max_depth=3, score=0.601, total=   0.7s


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  6.2min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1),
                   n_iter=100,
                   param_distributions={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10,
                                                      11, 12],
                                        'max_features': [2, 3, 4, 5, 6, 7, 8,
                                                         9],
                                        'min_samples_leaf': [10, 12, 14, 15, 16,
                                                             18],
                                        'min_samples_split': [10, 12, 14, 15,
                                                              16, 18],
                                        'n_estimators': [80, 83, 84, 85, 87, 89,
                                                         90, 91, 93]},
                   scoring='f1', verbose=3)

In [88]:
random_search.best_estimator_

RandomForestClassifier(max_depth=12, max_features=5, min_samples_leaf=12,
                       min_samples_split=10, n_estimators=87, random_state=1)

In [89]:
random_search.best_params_

{'n_estimators': 87,
 'min_samples_split': 10,
 'min_samples_leaf': 12,
 'max_features': 5,
 'max_depth': 12}

In [90]:
model = random_search.best_estimator_

#Predicciones set de train
preds = model.predict(X)

rmse = np.sqrt(mean_squared_error(train['target'], preds))
print(f'RMSE: %f' % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(train['target'], preds))

RMSE: 0.507658
F1_score:  0.6586638830897703


In [96]:
#AHORA PRUEBO DENUEVO PERO PARA UN REGRESSOR
params = {
 "max_depth"        : [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 "n_estimators"     : [80, 83, 84, 85, 87, 89, 90, 91, 93],
 "min_samples_split": [10, 12, 14, 15, 16, 18],
 "min_samples_leaf" : [10, 12, 14, 15, 16, 18],
 "max_features"     : [2, 3, 4, 5, 6, 7, 8, 9]
} 

model = RandomForestRegressor(random_state=1)

random_search = RandomizedSearchCV(model ,param_distributions = params, cv=5, verbose=3, n_iter=100)
random_search.fit(X, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] n_estimators=93, min_samples_split=15, min_samples_leaf=18, max_features=2, max_depth=10 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=93, min_samples_split=15, min_samples_leaf=18, max_features=2, max_depth=10, score=0.165, total=   0.5s
[CV] n_estimators=93, min_samples_split=15, min_samples_leaf=18, max_features=2, max_depth=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  n_estimators=93, min_samples_split=15, min_samples_leaf=18, max_features=2, max_depth=10, score=0.141, total=   0.5s
[CV] n_estimators=93, min_samples_split=15, min_samples_leaf=18, max_features=2, max_depth=10 


...

[CV]  n_estimators=89, min_samples_split=18, min_samples_leaf=18, max_features=4, max_depth=7, score=0.168, total=   0.5s
[CV] n_estimators=89, min_samples_split=18, min_samples_leaf=18, max_features=4, max_depth=7 
[CV]  n_estimators=89, min_samples_split=18, min_samples_leaf=18, max_features=4, max_depth=7, score=0.138, total=   0.5s
[CV] n_estimators=89, min_samples_split=18, min_samples_leaf=18, max_features=4, max_depth=7 
[CV]  n_estimators=89, min_samples_split=18, min_samples_leaf=18, max_features=4, max_depth=7, score=0.150, total=   0.5s
[CV] n_estimators=89, min_samples_split=18, min_samples_leaf=18, max_features=4, max_depth=7 
[CV]  n_estimators=89, min_samples_split=18, min_samples_leaf=18, max_features=4, max_depth=7, score=0.169, total=   0.5s
[CV] n_estimators=89, min_samples_split=18, min_samples_leaf=18, max_features=4, max_depth=7 
[CV]  n_estimators=89, min_samples_split=18, min_samples_leaf=18, max_features=4, max_depth=7, score=0.226, total=   0.5s
[CV] n_estimat

[CV]  n_estimators=85, min_samples_split=18, min_samples_leaf=18, max_features=9, max_depth=7, score=0.167, total=   0.8s
[CV] n_estimators=85, min_samples_split=18, min_samples_leaf=18, max_features=9, max_depth=7 
[CV]  n_estimators=85, min_samples_split=18, min_samples_leaf=18, max_features=9, max_depth=7, score=0.228, total=   0.8s
[CV] n_estimators=83, min_samples_split=12, min_samples_leaf=16, max_features=9, max_depth=10 
[CV]  n_estimators=83, min_samples_split=12, min_samples_leaf=16, max_features=9, max_depth=10, score=0.161, total=   1.0s
[CV] n_estimators=83, min_samples_split=12, min_samples_leaf=16, max_features=9, max_depth=10 
[CV]  n_estimators=83, min_samples_split=12, min_samples_leaf=16, max_features=9, max_depth=10, score=0.125, total=   0.9s
[CV] n_estimators=83, min_samples_split=12, min_samples_leaf=16, max_features=9, max_depth=10 
[CV]  n_estimators=83, min_samples_split=12, min_samples_leaf=16, max_features=9, max_depth=10, score=0.129, total=   0.9s
[CV] n_e

[CV]  n_estimators=85, min_samples_split=16, min_samples_leaf=15, max_features=3, max_depth=12, score=0.132, total=   0.5s
[CV] n_estimators=85, min_samples_split=16, min_samples_leaf=15, max_features=3, max_depth=12 
[CV]  n_estimators=85, min_samples_split=16, min_samples_leaf=15, max_features=3, max_depth=12, score=0.147, total=   0.5s
[CV] n_estimators=85, min_samples_split=16, min_samples_leaf=15, max_features=3, max_depth=12 
[CV]  n_estimators=85, min_samples_split=16, min_samples_leaf=15, max_features=3, max_depth=12, score=0.174, total=   0.6s
[CV] n_estimators=85, min_samples_split=16, min_samples_leaf=15, max_features=3, max_depth=12 
[CV]  n_estimators=85, min_samples_split=16, min_samples_leaf=15, max_features=3, max_depth=12, score=0.228, total=   0.5s


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  5.5min finished


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=1),
                   n_iter=100,
                   param_distributions={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10,
                                                      11, 12],
                                        'max_features': [2, 3, 4, 5, 6, 7, 8,
                                                         9],
                                        'min_samples_leaf': [10, 12, 14, 15, 16,
                                                             18],
                                        'min_samples_split': [10, 12, 14, 15,
                                                              16, 18],
                                        'n_estimators': [80, 83, 84, 85, 87, 89,
                                                         90, 91, 93]},
                   verbose=3)

In [97]:
random_search.best_estimator_

RandomForestRegressor(max_depth=9, max_features=3, min_samples_leaf=12,
                      min_samples_split=16, n_estimators=87, random_state=1)

In [98]:
random_search.best_params_

{'n_estimators': 87,
 'min_samples_split': 16,
 'min_samples_leaf': 12,
 'max_features': 3,
 'max_depth': 9}

In [99]:
model = random_search.best_estimator_

#Predicciones set de train
preds = model.predict(X)

rmse = np.sqrt(mean_squared_error(train['target'], preds))
print(f'RMSE: %f' % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(train['target'], preds))
#puedo ver que dió mejor resultado el classifier

RMSE: 0.427742
F1_score:  0.6237347294938917


**Pruebo usando Grid-Search como sugiere el apunte para encontrar la cantidad de atributos por árbol usando la precisión OOB**

In [103]:
from sklearn.model_selection import GridSearchCV

params = {
 "max_depth"        : [7, 8, 9, 10, 11, 12],
 "n_estimators"     : [84, 85, 87, 89, 90],
 "min_samples_split": [10, 12, 14],
 "min_samples_leaf" : [10, 12, 14],
 "max_features"     : [5, 6, 7, 8]
} 


model = RandomForestClassifier(random_state = 1, oob_score = True)

grid_search = GridSearchCV(model, param_grid = params, cv = 5, verbose = 3)
grid_search.fit(X, y)

Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] max_depth=7, max_features=5, min_samples_leaf=10, min_samples_split=10, n_estimators=84 
[CV]  max_depth=7, max_features=5, min_samples_leaf=10, min_samples_split=10, n_estimators=84, score=0.696, total=   1.1s
[CV] max_depth=7, max_features=5, min_samples_leaf=10, min_samples_split=10, n_estimators=84 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV]  max_depth=7, max_features=5, min_samples_leaf=10, min_samples_split=10, n_estimators=84, score=0.674, total=   1.0s
[CV] max_depth=7, max_features=5, min_samples_leaf=10, min_samples_split=10, n_estimators=84 


...

[CV]  max_depth=12, max_features=8, min_samples_leaf=12, min_samples_split=14, n_estimators=90, score=0.653, total=   1.5s
[CV] max_depth=12, max_features=8, min_samples_leaf=12, min_samples_split=14, n_estimators=90 
[CV]  max_depth=12, max_features=8, min_samples_leaf=12, min_samples_split=14, n_estimators=90, score=0.683, total=   1.3s
[CV] max_depth=12, max_features=8, min_samples_leaf=12, min_samples_split=14, n_estimators=90 
[CV]  max_depth=12, max_features=8, min_samples_leaf=12, min_samples_split=14, n_estimators=90, score=0.731, total=   1.3s
[CV] max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=10, n_estimators=84 
[CV]  max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=10, n_estimators=84, score=0.698, total=   1.4s
[CV] max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=10, n_estimators=84 
[CV]  max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=10, n_estimators=84, score=0.672, total=   1.2s
[CV] 

[CV]  max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=12, n_estimators=87, score=0.697, total=   1.1s
[CV] max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=12, n_estimators=87 
[CV]  max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=12, n_estimators=87, score=0.671, total=   1.2s
[CV] max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=12, n_estimators=87 
[CV]  max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=12, n_estimators=87, score=0.657, total=   1.1s
[CV] max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=12, n_estimators=87 
[CV]  max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=12, n_estimators=87, score=0.683, total=   1.2s
[CV] max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=12, n_estimators=87 
[CV]  max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=12, n_estimators=87, score=0.730, total=   1.1s
[CV] 

[CV]  max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=14, n_estimators=90, score=0.683, total=   1.2s
[CV] max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=14, n_estimators=90 
[CV]  max_depth=12, max_features=8, min_samples_leaf=14, min_samples_split=14, n_estimators=90, score=0.729, total=   1.2s


[Parallel(n_jobs=1)]: Done 5400 out of 5400 | elapsed: 98.8min finished


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(oob_score=True, random_state=1),
             param_grid={'max_depth': [7, 8, 9, 10, 11, 12],
                         'max_features': [5, 6, 7, 8],
                         'min_samples_leaf': [10, 12, 14],
                         'min_samples_split': [10, 12, 14],
                         'n_estimators': [84, 85, 87, 89, 90]},
             verbose=3)

In [104]:
grid_search.best_estimator_

RandomForestClassifier(max_depth=12, max_features=5, min_samples_leaf=14,
                       min_samples_split=10, n_estimators=90, oob_score=True,
                       random_state=1)

In [105]:
grid_search.best_params_

{'max_depth': 12,
 'max_features': 5,
 'min_samples_leaf': 14,
 'min_samples_split': 10,
 'n_estimators': 90}

In [108]:
model = grid_search.best_estimator_

In [109]:
#Predicciones set de train
preds = model.predict(X)


rmse = np.sqrt(mean_squared_error(train['target'], preds))
print(f'RMSE: %f' % (rmse))
preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(train['target'], preds))
print("Out of bounds score: ", model.oob_score_)

RMSE: 0.510625
F1_score:  0.654120926990765
Out of bounds score:  0.698147904899514


In [110]:
model.feature_importances_

array([0.17905675, 0.1055725 , 0.3430882 , 0.05244098, 0.11436497,
       0.03499773, 0.01285612, 0.05768364, 0.09993912])

In [111]:
X

,length,amount_words,amount_keywords_in_tweet,amount_locations_in_tweet,amount_urls,amount_users,amount_hashtags,len_hash_over_text,has_url
0,69,7,1,0,0,0,1,0.144928,0
1,38,7,2,1,0,0,0,0.000000,0
2,133,11,1,1,0,0,0,0.000000,0
3,65,6,3,2,0,0,1,0.138462,0
4,88,9,3,1,0,0,2,0.170455,0
...,...,...,...,...,...,...,...,...,...
7608,83,8,2,0,1,0,0,0.000000,1
7609,125,11,1,1,0,2,0,0.000000,0
7610,65,3,1,0,1,0,0,0.000000,1
7611,137,14,5,2,0,0,0,0.000000,0
